## create the DataBase

In [ ]:
import pandas as pd
import sqlite3

# kaggle data link:
link = 'https://www.kaggle.com/martj42/international-football-results-from-1872-to-2017'
# to the path below download from kaggle 3 csv files: results.csv, goalscorers.csv, shootouts.csv
path = 'C:/Users/Mark Rozenberg/Foobal-Climate/Data/footbal/'
# create sql database with 3 fotbal tables:
conn = sqlite3.connect('C:/Users/Mark Rozenberg/Foobal-Climate/Data/Main_DB.db')
pd.read_csv(path + 'results.csv').to_sql('results', conn)
pd.read_csv(path + 'goalscorers.csv').to_sql('goalscorers', conn)
pd.read_csv(path + 'shootouts.csv').to_sql('shootouts', conn)
conn.commit()

In [2]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('C:/Users/Mark Rozenberg/Foobal-Climate/Data/Main_DB.db')

### updating results table

In [3]:
from unidecode import unidecode

# Fetch distinct cities from the results table
cities = pd.read_sql_query('SELECT DISTINCT city FROM results', conn)

# Apply unidecode to each city
cities['city_v2'] = cities['city'].apply(unidecode)

# Update the results table with the new city names
for index, row in cities.iterrows():
    conn.execute('''
    UPDATE results
    SET city = ?
    WHERE city = ?
    ''', (row['city_v2'], row['city']))

conn.commit()

In [9]:
# simple overview of the data:
fuf = pd.read_sql_query('SELECT * FROM results limit 5', conn)

In [8]:
test = print(pd.read_sql_query('''
SELECT tournament,
count(*) as games
FROM results
where tournament like '%Leag%'
group by 1
order by 2 desc
''', conn))

                              tournament  games
0                    UEFA Nations League    526
1                CONCACAF Nations League    351
2  CONCACAF Nations League qualification     68


In [10]:
test = print(pd.read_sql_query('''
SELECT city,
count(*) as games
FROM results
group by 1
order by 2 desc
''', conn))

              city  games
0     Kuala Lumpur    730
1          Bangkok    573
2             Doha    546
3           London    428
4         Budapest    425
...            ...    ...
2067      Acarigua      1
2068      Acapulco      1
2069      Aberdare      1
2070      Abeokuta      1
2071         Aarau      1

[2072 rows x 2 columns]


In [34]:
# Query to list all tables in the database
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='view';", conn)

,name
0,top_5_goal_scorers
1,goal_scorers_summary


In [32]:
conn.execute('''drop view if exists goal_scorers_summary''')
conn.commit()

In [29]:
pd.read_sql_query(''' select * from goalscorers limit 3''', conn)

,index,date,home_team,away_team,team,scorer,minute,own_goal,penalty
0,0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,0,0
1,1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,0,0
2,2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,0,0


In [30]:
pd.read_sql_query(''' select * from results limit 3''', conn)

,index,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,noaa_country_code,location_id,mindate,maxdate,date_ts,mindate_ts,maxdate_ts,valid
0,0,1872-11-30 00:00:00,Scotland,England,0,0,Friendly,Glasgow,United Kingdom,0,UK,CITY:UK000006,1914-01-01,2024-08-31,1872-11-30,1914-01-01,2024-08-31,0
1,1,1873-03-08 00:00:00,England,Scotland,4,2,Friendly,London,United Kingdom,0,UK,CITY:UK000009,1904-02-01,2024-09-24,1873-03-08,1904-02-01,2024-09-24,0
2,2,1874-03-07 00:00:00,Scotland,England,2,1,Friendly,Glasgow,United Kingdom,0,UK,CITY:UK000006,1914-01-01,2024-08-31,1874-03-07,1914-01-01,2024-08-31,0


In [7]:
pd.read_sql_query(''' select * from results limit 3''', conn)

,index,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,noaa_country_code,location_id,mindate,maxdate,date_ts,mindate_ts,maxdate_ts,valid,location_date_id
0,0,1872-11-30 00:00:00,Scotland,England,0,0,Friendly,Glasgow,United Kingdom,0,UK,CITY:UK000006,1914-01-01,2024-08-31,1872-11-30,1914-01-01,2024-08-31,0,CITY:UK000006_1872-11-30
1,1,1873-03-08 00:00:00,England,Scotland,4,2,Friendly,London,United Kingdom,0,UK,CITY:UK000009,1904-02-01,2024-09-24,1873-03-08,1904-02-01,2024-09-24,0,CITY:UK000009_1873-03-08
2,2,1874-03-07 00:00:00,Scotland,England,2,1,Friendly,Glasgow,United Kingdom,0,UK,CITY:UK000006,1914-01-01,2024-08-31,1874-03-07,1914-01-01,2024-08-31,0,CITY:UK000006_1874-03-07


In [4]:
from google.cloud import storage

def list_buckets():
    storage_client = storage.Client()
    buckets = list(storage_client.list_buckets())
    print("Buckets:")
    for bucket in buckets:
        print(bucket.name)

list_buckets()

Buckets:
footbal_climate_data
ltint_forecast


In [5]:
import pandas as pd
from google.cloud import storage
import sqlite3

def export_view_to_csv_and_upload(view_name):
    # Load the data into a DataFrame
    df = pd.read_sql(f"SELECT * FROM {view_name}", conn)

    csv_file_path = view_name + '.csv'
    print(csv_file_path)
    # Save the DataFrame to a CSV file
    df.to_csv(csv_file_path, index=False)

    # Upload the CSV file to GCP
    storage_client = storage.Client()
    bucket = storage_client.bucket("footbal_climate_data")
    blob = bucket.blob(csv_file_path)
    # Delete the blob if it already exists
    if blob.exists():
        blob.delete()

    blob.upload_from_filename(csv_file_path)

    print(f"File {csv_file_path} uploaded to {csv_file_path}.")

# Example usage
export_view_to_csv_and_upload('results')

results.csv
File results.csv uploaded to results.csv.


In [12]:
# updating the table in BigQuery:
from google.cloud import bigquery
from google.cloud import storage

client = bigquery.Client()
table_id = "footbal-climate.footbal_climate_db.results"

# Get the blob from GCS
storage_client = storage.Client()
storage_client.bucket('footbal_climate_data').blob('results.csv').download_to_filename('/tmp/results.csv')


job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)
with open('/tmp/results.csv', 'rb') as source_file:
    load_job = client.load_table_from_file(source_file, table_id, job_config=job_config)

load_job.result()
print("Table overwritten successfully.")


Table overwritten successfully.


In [10]:
conn.execute('''DROP VIEW IF EXISTS team_stats''')
conn.commit()

In [11]:
pd.read_sql_query(''' select * from goalscorers limit 3''', conn)

,index,date,home_team,away_team,team,scorer,minute,own_goal,penalty
0,0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,0,0
1,1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,0,0
2,2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,0,0


In [ ]:
scorers_df = pd.read_sql_query('''
SELECT scorer, team, strftime('%Y', date) as year,
COUNT(*) as goals,
avg(minute) as avg_minute,
sum(case when minute <= 45 then 1 else 0 end) as goals_first_half,
sum(case when minute > 45 then 1 else 0 end) as goals_second_half,
sum(own_goal) as own_goals,
sum(penalty) as penalties
FROM goalscorers
where scorer is not null
GROUP BY 1,2,3
ORDER BY year DESC
'''
, conn)

In [3]:
pd.read_sql_query('''select * from goalscorers limit 3''', conn)

,index,date,home_team,away_team,team,scorer,minute,own_goal,penalty
0,0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,0,0
1,1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,0,0
2,2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,0,0
